In [10]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from fastbm25 import fastbm25
from joblib import Parallel, delayed

In [11]:
path = Path.home() / 'PycharmProjects/ML-learning-journey/nlp/text_retrieval/data/'
df = pd.read_csv(path / 'data.csv')
df['token'] = df['company_edit_name'].str.split()
df

,Line Number,Company Name,Company CIK Key,company_edit_name,token
0,1,!J INC,1438823,!j inc,"[!j, inc]"
1,2,"#1 A LIFESAFER HOLDINGS, INC.",1509607,"#1 a lifesafer holdings, inc.","[#1, a, lifesafer, holdings,, inc.]"
2,3,#1 ARIZONA DISCOUNT PROPERTIES LLC,1457512,#1 arizona discount properties llc,"[#1, arizona, discount, properties, llc]"
3,4,#1 PAINTBALL CORP,1433777,#1 paintball corp,"[#1, paintball, corp]"
4,5,$ LLC,1427189,$ llc,"[$, llc]"
...,...,...,...,...,...
49995,49996,BABB DOUGLAS J,1190359,babb douglas j,"[babb, douglas, j]"
49996,49997,BABB HENRY C,1193948,babb henry c,"[babb, henry, c]"
49997,49998,BABB INTERNATIONAL INC,1139504,babb international inc,"[babb, international, inc]"
49998,49999,BABB JACK J,1280368,babb jack j,"[babb, jack, j]"


In [12]:
corpus = df['token'].values.tolist()
bm25 = BM25Okapi(corpus)
model_fast = fastbm25(corpus)

In [13]:
queries = '09'
meta = {}
scores = bm25.get_top_n(queries.split(), corpus, 5)
meta[queries] = list(map(' '.join, scores))
meta

{'09': ['09 proud citizen/marialua, llc',
  '09 flashy bull/laelia, llc',
  '09 silver train/alotofappeal, llc',
  '09 flashy bull/desert sunrise, llc',
  "09 city zip/capote's tune, llc"]}

In [14]:
queries = '09'
meta = {}
scores = model_fast.top_k_sentence(queries.split(), 5)
meta[queries] = scores
meta

{'09': [(['09', 'flashy', 'bull/laelia,', 'llc'], 142, 7.64),
  (['09', 'proud', 'citizen/marialua,', 'llc'], 148, 7.64),
  (['09', 'silver', 'train/alotofappeal,', 'llc'], 154, 7.64),
  (['09', 'awesome', 'again/blind', 'canyon,', 'llc'], 133, 6.87),
  (['09', 'bellamy', 'road/forever', 'dixie,', 'llc'], 134, 6.87)]}

In [15]:
scores[0]

(['09', 'flashy', 'bull/laelia,', 'llc'], 142, 7.64)

In [16]:
def run_func(queries, corpus, top_k=5):
    meta = {}
    scores = bm25.get_top_n(queries.split(), corpus, top_k)
    meta[queries] = list(map(' '.join, scores))
    return meta

In [17]:
num_jobs = -1
queries = ['09', '08', 'flashy']
results = Parallel(n_jobs=num_jobs)(delayed(run_func)(query, corpus, 5) for query in tqdm(queries))

100%|██████████| 3/3 [00:00<00:00, 5904.70it/s]


In [18]:
tmp = pd.concat([pd.DataFrame(i.items()) for i in results])
tmp = tmp.explode(1)
tmp

,0,1
0,09,"09 proud citizen/marialua, llc"
0,09,"09 flashy bull/laelia, llc"
0,09,"09 silver train/alotofappeal, llc"
0,09,"09 flashy bull/desert sunrise, llc"
0,09,"09 city zip/capote's tune, llc"
0,08,"08 flatter/glamorama, llc"
0,08,"08 johar/zawaaya, llc"
0,08,"08 da stoops/jathibiyah, llc"
0,08,"08 smoke glacken/minado, llc"
0,08,"08 grand slam/feminine, llc"
